<a href="https://colab.research.google.com/github/Testgitchhub/Smart-Image-Interpretation-Chatbot-With-Speech-Synthesis-And-Image-Generation-/blob/main/Smart%20Image%20Chatbot%20with%20Speech%20Synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utils.py
import os
import torch
from PIL import Image
import pytesseract
import language_tool_python
from gtts import gTTS
from transformers import BlipProcessor, BlipForConditionalGeneration
import cohere
import uuid

In [ ]:
# Load models once
device = "cuda" if torch.cuda.is_available() else "cpu"
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

cohere_api_key = os.getenv("COHERE_API_KEY")
co = cohere.Client(cohere_api_key)
tool = language_tool_python.LanguageTool('en-US')
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = blip_processor(image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    caption = blip_processor.decode(out[0], skip_special_tokens=True)
    return caption

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text.strip()

def is_grammatically_correct(text):
    matches = tool.check(text)
    return len(matches) == 0

def query_cohere(caption, ocr_text, user_question):
    if is_grammatically_correct(ocr_text):
        prompt = f"Image Caption: {caption}\nExtracted Text: {ocr_text}\nUser Question: {user_question}\nAnswer:"
    else:
        prompt = f"Image Caption: {caption}\nUser Question: {user_question}\nAnswer:"
    response = co.generate(
        model="command-xlarge",
        prompt=prompt,
        max_tokens=100,
        temperature=0.75
    )
    return response.generations[0].text.strip()

def speak_text(text):
    filename = f"audio_{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=text, lang='en', slow=False)
    tts.save(filename)
    return filename
